In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [11]:
df = pd.read_csv('./data/train.csv')
df

,VERDICT,PARAMETER_1,PARAMETER_2,PARAMETER_3,PARAMETER_4,PARAMETER_5,PARAMETER_6,PARAMETER_7,PARAMETER_8,PARAMETER_9
0,1,39353,85475,117961,118300,123472,117905,117906,290919,117908
1,1,17183,1540,117961,118343,123125,118536,118536,308574,118539
2,1,36724,14457,118219,118220,117884,117879,267952,19721,117880
3,1,36135,5396,117961,118343,119993,118321,240983,290919,118322
4,1,42680,5905,117929,117930,119569,119323,123932,19793,119325
...,...,...,...,...,...,...,...,...,...,...
32764,1,23497,16971,117961,118300,119993,118321,240983,290919,118322
32765,1,25139,311198,91261,118026,122392,121143,173805,249618,121145
32766,1,34924,28805,117961,118327,120299,124922,152038,118612,124924
32767,1,80574,55643,118256,118257,117945,280788,280788,292795,119082


In [12]:
df.isnull().any().sum()

0

In [13]:
X = df.drop(['VERDICT'], axis=1)
y = df.get(['VERDICT'])


In [14]:
print(X.shape)
print(y.shape)

(32769, 9)
(32769, 1)


In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=123)
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [16]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(9,)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32)

Epoch 1/50
687/687 [==============================] - 3s 4ms/step - loss: 0.2461 - accuracy: 0.9411
Epoch 2/50
687/687 [==============================] - 2s 3ms/step - loss: 0.2240 - accuracy: 0.9416
Epoch 3/50
687/687 [==============================] - 2s 2ms/step - loss: 0.2199 - accuracy: 0.9416
Epoch 4/50
687/687 [==============================] - 2s 2ms/step - loss: 0.2184 - accuracy: 0.9414
Epoch 5/50
687/687 [==============================] - 2s 3ms/step - loss: 0.2179 - accuracy: 0.9416
Epoch 6/50
687/687 [==============================] - 2s 3ms/step - loss: 0.2169 - accuracy: 0.9416
Epoch 7/50
687/687 [==============================] - 2s 3ms/step - loss: 0.2168 - accuracy: 0.9416
Epoch 8/50
687/687 [==============================] - 2s 3ms/step - loss: 0.2161 - accuracy: 0.9415
Epoch 9/50
687/687 [==============================] - 2s 3ms/step - loss: 0.2149 - accuracy: 0.9417
Epoch 10/50
687/687 [==============================] - 2s 2ms/step - loss: 0.2146 - accuracy: 0.9416

In [17]:
predictions = model.predict(X_test)

# Convert predictions to binary data
binary_predictions = (predictions > 0.5).astype(int)
accuracy = tf.keras.metrics.Accuracy()
accuracy.update_state(y_test, binary_predictions)
print('Accuracy:', accuracy.result().numpy())

338/338 [==============================] - 0s 1ms/step
Accuracy: 0.94257444


In [18]:
X_submit = pd.read_csv('./data/test.csv').drop(['Id'], axis=1)
predictions = model.predict(X_submit)

# Convert predictions to binary labels
binary_predictions = (predictions > 0.5).astype(int)
c =0
for i in binary_predictions:
    if i == 0:
        c+=1
print('No. of zeros: ',c)
data = { 'Id' : np.arange(1,58922),
        'VERDICT':  np.ndarray.flatten(binary_predictions)}
ans = pd.DataFrame(data)
ans.to_csv('predictions.csv', index=False)

1842/1842 [==============================] - 2s 1ms/step
No. of zeros:  25375
